In [ ]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm_notebook

import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_path = 'birds_rev2/train'
test_path = 'birds_rev2/test'

train_data = datasets.ImageFolder(train_path, transform= transform)
test_data = datasets.ImageFolder(test_path, transform= transform)

batch = 32

train_loader = DataLoader(train_data, batch_size=batch, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch, shuffle=True)

def train(epoches, model, data_loader, optimizer, criterion, dataset_size):
    best_loss = 0.00065401958121375
    for epoch in range(epoches):
        running_loss = []
        running_acc = 0
        for X, y in tqdm_notebook(data_loader):

            X, y = X.to(device), y.to(device)

            y_pred = model(X)

            loss = criterion(y_pred, y)

            _, pred = torch.max(y_pred, 1)

            running_acc += torch.sum(pred == y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss.append(loss.item())
        print('EPOCH: {}\t LOSS: {}\t ACC: {}'.format(epoch + 1, np.mean(running_loss), running_acc/dataset_size * 100))
        mean = np.mean(running_loss)
        if mean < best_loss:
            print('Saving Model...')
            best_loss = mean
            torch.save(model.state_dict(),'/content/drive/MyDrive/weights/birds.pt')

model = models.resnet50(pretrained=True)
output_size = len(train_data.classes)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, out_features=output_size)

try:
    model.load_state_dict(torch.load('/content/drive/MyDrive/weights/birds.pt'))
except:
    pass
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

train(100, model, train_loader, optimizer, criterion, len(train_data))

In [ ]:
def eval(model, data_loader, dataset_size):

    running_acc = 0
    model.eval()
    for X, y in data_loader:

        X, y = X.to(device), y.to(device)

        y_pred = model(X)
        loss = criterion(y_pred, y)

        _, pred = torch.max(y_pred, 1)

        running_acc += torch.sum(pred == y)
    print('ACC: {}'.format(running_acc/dataset_size * 100))
    model.train()

eval(model, test_loader, len(test_data))

ACC: 97.16363525390625
